In [1]:
import splusdata
import getpass
import pandas as pd
from astropy.table import Table, vstack
import numpy as np
from pathlib import Path

In [2]:
ROOT_PATH = Path("../Final-list/") 

In [3]:
# Connecting with SPLUS database
username = input(prompt="Login: ")
password = getpass.getpass("Password: ")
conn = splusdata.connect(username, password)

Login: Luis
Password: ········
You have access to internal data


In [23]:
# Reading the csv table
df = pd.read_csv(ROOT_PATH / 'Halpha-emiters-splusdr3.csv')
print(df)

              Field                         ID          RA        DEC  \
0        HYDRA-0011     iDR3.HYDRA-0011.012297  150.477365 -24.377933   
1        HYDRA-0011     iDR3.HYDRA-0011.024314  150.343054 -24.184322   
2        HYDRA-0011     iDR3.HYDRA-0011.032883  150.382960 -24.025783   
3        HYDRA-0011     iDR3.HYDRA-0011.044169  151.440282 -23.830519   
4        HYDRA-0011     iDR3.HYDRA-0011.067988  150.574490 -23.181332   
...             ...                        ...         ...        ...   
8441  STRIPE82-0169  iDR3.STRIPE82-0169.004840  358.422066  -1.317507   
8442  STRIPE82-0169  iDR3.STRIPE82-0169.008103  358.799382  -1.159854   
8443  STRIPE82-0169  iDR3.STRIPE82-0169.028275  358.942267  -0.395210   
8444  STRIPE82-0169  iDR3.STRIPE82-0169.033427  358.366835   0.031941   
8445  STRIPE82-0169  iDR3.STRIPE82-0169.037778  357.803175  -0.196838   

           FWHM  ISOarea  KRON_RADIUS     MU_MAX  nDet_PStotal  PhotoFlagDet  \
0      2.820643       11     2.813835  17.6

In [24]:
#Selecting columns
col = ["Field", "ID", "RA", "DEC"]
df_new = df[col]
df_new

,Field,ID,RA,DEC
0,HYDRA-0011,iDR3.HYDRA-0011.012297,150.477365,-24.377933
1,HYDRA-0011,iDR3.HYDRA-0011.024314,150.343054,-24.184322
2,HYDRA-0011,iDR3.HYDRA-0011.032883,150.382960,-24.025783
3,HYDRA-0011,iDR3.HYDRA-0011.044169,151.440282,-23.830519
4,HYDRA-0011,iDR3.HYDRA-0011.067988,150.574490,-23.181332
...,...,...,...,...
8441,STRIPE82-0169,iDR3.STRIPE82-0169.004840,358.422066,-1.317507
8442,STRIPE82-0169,iDR3.STRIPE82-0169.008103,358.799382,-1.159854
8443,STRIPE82-0169,iDR3.STRIPE82-0169.028275,358.942267,-0.395210
8444,STRIPE82-0169,iDR3.STRIPE82-0169.033427,358.366835,0.031941


In [25]:
Query = f"""SELECT det.ID, det.ra, det.dec, det.CLASS_STAR 
                FROM TAP_UPLOAD.upload AS upl 
                LEFT OUTER JOIN "idr3"."detection_image" AS det ON upl.ID = det.ID"""

In [26]:
n = int(len(df_new) / 5000.) + 1
df_ = [] # list
j = 0 # counter
d = {} # empty
for i in range(n):
    j += 1  
    df_.append(df_new.iloc[5000*i:5000*j])

In [27]:
merged_table_list = []
for a in range(2):
    results = conn.query(Query, df_[a])
    merged_table_list.append(results)

Cutting to the first 6000 objects!
finished
Cutting to the first 6000 objects!
finished


In [28]:
merged_table_list

[<Table length=5000>
            ID                    RA                 DEC         CLASS_STAR
         bytes24               float64             float64        float32  
 ------------------------ ------------------ ------------------- ----------
 iDR3.HYDRA-0011.012297     150.477365078257 -24.377932572283253 0.87623084
 iDR3.HYDRA-0011.024314   150.34305416196275 -24.184322408374864  0.8254107
 iDR3.HYDRA-0011.032883    150.3829599194086 -24.025782586370628  0.9806283
 iDR3.HYDRA-0011.044169   151.44028216685064  -23.83051899079528  0.9992462
 iDR3.HYDRA-0011.067988   150.57449037365305 -23.181331797832584  0.8956589
 iDR3.HYDRA-0011.073322   149.99160585251383   -23.1797200452264 0.91642165
 iDR3.HYDRA-0011.075548   150.44330135993783 -23.173862289410675 0.97497237
 iDR3.HYDRA-0012.007283   151.59563071852426  -24.48508161702202  0.8973185
                      ...                ...                 ...        ...
 iDR3.SPLUS-s24s14.014612 21.251050178292942 -31.081182425794243 0.

In [29]:
# Merging all result astropy tables 
merged_table = vstack(merged_table_list)
len(merged_table)

8446

In [30]:
df["CLASS_STAR"] = merged_table["CLASS_STAR"]

In [31]:
df

,Field,ID,RA,DEC,FWHM,ISOarea,KRON_RADIUS,MU_MAX,nDet_PStotal,PhotoFlagDet,...,e_r_PStotal,e_J0660_PStotal,e_i_PStotal,e_J0861_PStotal,e_z_PStotal,Label_hier,Label_hdbscan,P(Blue),P(Red),CLASS_STAR
0,HYDRA-0011,iDR3.HYDRA-0011.012297,150.477365,-24.377933,2.820643,11,2.813835,17.650438,12.0,0,...,0.062504,0.042892,0.091060,0.133724,0.153265,0.0,-1.0,0.208840,0.042880,0.876231
1,HYDRA-0011,iDR3.HYDRA-0011.024314,150.343054,-24.184322,2.395437,10,3.194570,17.982782,10.0,0,...,0.090772,0.068010,0.091906,0.243493,0.180905,NaN,NaN,NaN,NaN,0.825411
2,HYDRA-0011,iDR3.HYDRA-0011.032883,150.382960,-24.025783,1.988281,67,3.132211,14.042880,12.0,2,...,0.007069,0.007504,0.007045,0.017350,0.009810,0.0,0.0,0.971265,0.028735,0.980628
3,HYDRA-0011,iDR3.HYDRA-0011.044169,151.440282,-23.830519,2.058785,60,3.434575,14.659185,12.0,0,...,0.008902,0.010606,0.008999,0.022395,0.013073,0.0,0.0,0.836676,0.036995,0.999246
4,HYDRA-0011,iDR3.HYDRA-0011.067988,150.574490,-23.181332,2.887644,37,1.820000,16.223858,9.0,0,...,0.027093,0.030744,0.027158,0.058716,0.019140,1.0,-1.0,0.024175,0.042487,0.895659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8441,STRIPE82-0169,iDR3.STRIPE82-0169.004840,358.422066,-1.317507,42.047874,717,5.410226,16.574099,11.0,3,...,0.028457,0.022605,0.013941,0.056818,0.019170,1.0,-1.0,0.023206,0.040725,0.000342
8442,STRIPE82-0169,iDR3.STRIPE82-0169.008103,358.799382,-1.159854,2.716555,88,3.665439,14.348978,9.0,0,...,0.012314,0.006590,0.005344,0.008177,0.005035,1.0,-1.0,0.029796,0.051984,0.983464
8443,STRIPE82-0169,iDR3.STRIPE82-0169.028275,358.942267,-0.395210,2.963892,30,4.725904,16.711102,11.0,0,...,0.039238,0.038908,0.042814,0.112872,0.067588,0.0,0.0,0.926110,0.013910,0.678067
8444,STRIPE82-0169,iDR3.STRIPE82-0169.033427,358.366835,0.031941,4.231205,57,2.630473,15.495900,9.0,0,...,0.015825,0.011596,0.010325,0.024102,0.012136,1.0,-1.0,0.074960,0.204176,0.148657


In [32]:
df.to_csv(ROOT_PATH / "Halpha-emiters-splusdr3-class.csv", index=False)